In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
tf.__version__

## Importing Dataset

In [ ]:
dataset_training = pd.read_csv("/kaggle/input/sf-crime/train.csv.zip")
dataset_test = pd.read_csv("/kaggle/input/sf-crime/test.csv.zip")

In [ ]:
print(dataset_training.shape)
print(dataset_test.shape)

## Data Preprocessing

In [ ]:
dataset_training_new = dataset_training.drop(['Category', 'Descript', 'Resolution'], axis=1)
dataset_test_new = dataset_test.drop('Id', axis=1)
dataset_training_new['Dates'] = pd.to_datetime(dataset_training_new['Dates'])
dataset_test_new['Dates'] = pd.to_datetime(dataset_test_new['Dates'])

In [ ]:
plt.hist(dataset_training_new['Dates'].dt.minute)
plt.show()
plt.hist(dataset_training_new['Dates'].dt.day)
plt.show()
plt.hist(dataset_test_new['Dates'].dt.hour)
plt.show()
plt.hist(dataset_training_new['Dates'].dt.month)
plt.show()
plt.hist(dataset_training_new['Dates'].dt.year)
plt.show()
plt.hist(dataset_training_new['Dates'].dt.weekday)
plt.show()

In [ ]:
dataset_training_new['Year'] = dataset_training_new['Dates'].dt.year
dataset_test_new['Year'] = dataset_test_new['Dates'].dt.year
dataset_training_new['Month'] = dataset_training_new['Dates'].dt.month
dataset_test_new['Month'] = dataset_test_new['Dates'].dt.month
dataset_training_new['Day'] = dataset_training_new['Dates'].dt.day
dataset_test_new['Day'] = dataset_test_new['Dates'].dt.day
dataset_training_new['Hour'] = dataset_training_new['Dates'].dt.hour
dataset_test_new['Hour'] = dataset_test_new['Dates'].dt.hour
dataset_training_new['Minute'] = dataset_training_new['Dates'].dt.minute
dataset_test_new['Minute'] = dataset_test_new['Dates'].dt.minute
dataset_training_new['WeekDay'] = dataset_training_new['Dates'].dt.weekday
dataset_test_new['WeekDay'] = dataset_test_new['Dates'].dt.weekday
dataset_training_new.drop(['Dates', 'DayOfWeek', 'Address'], axis=1, inplace=True)
dataset_test_new.drop(['Dates', 'DayOfWeek', 'Address'], axis=1, inplace=True)


In [ ]:
print(dataset_test_new['PdDistrict'].unique())
print(dataset_training_new)

## One Hot Encode PdDistrict

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
X = np.array(ct.fit_transform(dataset_training_new))
X_submission = np.array(ct.transform(dataset_test_new))

In [ ]:
print(X.shape)
print(X_submission.shape)

In [ ]:
y = dataset_training['Category'].values
ohe = OneHotEncoder(sparse=False)
y = ohe.fit_transform(y.reshape(-1, 1))
y.shape

## Splitting Dataset into training and testing set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

## Feature Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Part - 2 Building and training the ANN

In [ ]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=200, activation='relu'))
ann.add(tf.keras.layers.Dense(units=200, activation='relu'))
ann.add(tf.keras.layers.Dense(units=200, activation='relu'))
ann.add(tf.keras.layers.Dense(units=200, activation='relu'))
ann.add(tf.keras.layers.Dense(units=39, activation='softmax'))
ann.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
ann.fit(X_train, y_train, batch_size=32, epochs=25, validation_data=(X_test, y_test))

# Part-3 Predicting the Result

In [ ]:
X_submission = sc.transform(X_submission)
y_pred = ann.predict(X_submission)
m = np.max(y_pred, axis=1).reshape(-1, 1)
predicted = np.array((y_pred == m), dtype='int32')
col_names = ohe.categories_[0]
df_submission = pd.DataFrame()
for i, entry in enumerate(col_names):
    df_submission[entry] = predicted[:,i]

df_submission['Id'] = dataset_test['Id']
df_submission.set_index('Id', inplace=True)
df_submission

In [ ]:
df_submission.to_csv('Submission.csv')